# Encoding 

In [123]:
%matplotlib inline

import os
import json
import time
import pickle
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [134]:
data = '/Users/skhederian/restaurant-health/the_final_countdown.csv'

names = [
    "_id",
    "restaurant_name",
    "address_full",
    "business_id",
    "categories",
    "city",
    "review_count",
    "inspection_date",
    "stars",
    "latitude",
    "longitude",
    "attributes",
    "open",
    "neighborhoods",
    "violations"
  ]

df = pd.read_csv(data, header=1, names=names)
df.head(10)

,_id,restaurant_name,address_full,business_id,categories,city,review_count,inspection_date,stars,latitude,longitude,attributes,open,neighborhoods,violations
0,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2008-05-07,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",7.0
1,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2008-05-15,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",7.0
2,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2009-03-12,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",11.0
3,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2010-04-12,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",15.0
4,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2010-06-09,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",1.0
5,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2010-06-21,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",1.0
6,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2010-11-03,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",2.0
7,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2011-04-19,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",4.0
8,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2011-05-09,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",4.0
9,ObjectId(5830680bf3f071f6de30b0f3),Beantown Pub,100 Tremont ST Boston 02108,fr2rdXAXq9NnAphpbPNELA,"[""Pubs"",""Bars"",""American (Traditional)"",""Night...",Boston,386,2011-11-22,3.0,42.357451,-71.060912,"{""Alcohol"":""full_bar"",""Noise Level"":""loud"",""Mu...",True,"[""Downtown""]",8.0


In [125]:
df.dtypes

_id                 object
restaurant_name     object
address_full        object
business_id         object
categories          object
city                object
review_count         int64
inspection_date     object
stars              float64
latitude           float64
longitude          float64
attributes          object
open                  bool
neighborhoods       object
violations         float64
dtype: object

In [126]:
import json

meta = {
    'target_names': list(df.violations.unique()),
    'feature_names': list(df.columns),
    'categorical_features': {
        column: list(df[column].unique())
        for column in df.columns
        if df[column].dtype == 'object'
    },
}

with open('data/meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

In [127]:
# Put the data into a bunch
from sklearn.datasets.base import Bunch
from sklearn.model_selection import train_test_split

def load_data(root='df'):
    # Load the meta data from the file
    with open('/Users/skhederian/restaurant-health/Data/meta.json', 'r') as f:
        meta = json.load(f)

    names = meta['feature_names']

    # Load the readme information
    with open('/Users/skhederian/restaurant-health/README.md', 'r') as f:
        readme = f.read()

    # Load the training and test data, skipping the bad row in the test data
    train, test = train_test_split(df, test_size = 0.2)

    # Return the bunch with the appropriate data chunked apart
    return Bunch(
        df = train[names[:-1]],
        target = train[names[-1]],
        data_test = test[names[:-1]],
        target_test = test[names[-1]],
        target_names = meta['target_names'],
        feature_names = meta['feature_names'],
        categorical_features = meta['categorical_features'],
        DESCR = readme,
    )

dataset = load_data()

In [128]:
# Create Multiple Column Encoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """

    def __init__(self, columns=None):
        self.columns  = columns
        self.encoders = None

    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns

        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns
        }
        return self

    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])

        return output

In [129]:
# Run Encoder
encoder = EncodeCategorical(columns = ['neighborhoods', 'city'])
newdata = encoder.fit_transform(dataset.df)

In [130]:
print(newdata)

                                      _id  \
1755   ObjectId(5830680bf3f071f6de30b0c5)   
17138  ObjectId(581151a5b6729e7e3134a3fb)   
16102  ObjectId(581151a7b6729e7e3134ed0c)   
4692   ObjectId(5830680bf3f071f6de30b1cf)   
21753  ObjectId(581151a6b6729e7e3134b542)   
43474  ObjectId(5830680bf3f071f6de30b299)   
24181  ObjectId(581151a4b6729e7e31343ff1)   
13639  ObjectId(581151a6b6729e7e3134b22d)   
23291  ObjectId(581151a5b6729e7e31347fc2)   
6462   ObjectId(5830680cf3f071f6de30b415)   
30754  ObjectId(581151a6b6729e7e3134e367)   
4679   ObjectId(5830680bf3f071f6de30b1bc)   
44087  ObjectId(5830680bf3f071f6de30b03a)   
41264  ObjectId(5830680bf3f071f6de30afd4)   
36232  ObjectId(5830680cf3f071f6de30b455)   
6799   ObjectId(5830680cf3f071f6de30b31c)   
29790  ObjectId(581151a4b6729e7e31345dbf)   
22022  ObjectId(581151a6b6729e7e3134df49)   
46116  ObjectId(5830680bf3f071f6de30af38)   
35558  ObjectId(5830680bf3f071f6de30b2e3)   
23875  ObjectId(581151a6b6729e7e3134c35e)   
40353  Obj

In [131]:
# Dummy Variable Option 
# df_neighborhood = pd.get_dummies(df['neighborhoods'], sparse=True)
# df_new = pd.concat([df, df_neighborhood], axis=1)
# df_new.describe()

In [133]:
# Run One Hot Encoder
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(newdata.neighborhoods)
enc.transform(newdata.neighborhoods).toarray()

ValueError: could not convert string to float: '{"Price Range":4,"Alcohol":"full_bar","Good For Dancing":false,"Noise Level":"average","Coat Check":false,"Has TV":true,"Happy Hour":false,"Music":{"live":false},"Outdoor Seating":false,"Parking":{"garage":true,"street":true,"validated":true,"lot":false,"valet":false},"Ambience":{"romantic":false,"intimate":false,"classy":false,"hipster":false,"divey":true,"touristy":false,"trendy":false,"upscale":false,"casual":false},"Smoking":"no","Accepts Credit Cards":true,"Good For Groups":true,"Wheelchair Accessible":false}'